<p style="font-family: Arial; font-size:3em;color:black;"> Session 4 - Stochastic Gradient Descent</p>

We will use the titanic dataset from Kaggle (https://www.kaggle.com/). This is a well-known dataset and we will use it for classification- if the passenger survived or passed away.

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
training_DF = pd.read_csv('titanic_dataset_GBC.csv')
training_DF.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
training_DF.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [13]:
training_DF['Age'].fillna(training_DF['Age'].mean(), inplace=True)

C:\Users\wayne\AppData\Local\Temp\ipykernel_2224\3895944149.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  training_DF['Age'].fillna(training_DF['Age'].mean(), inplace=True)


In [15]:
training_DF.drop('Cabin',axis=1,inplace=True)

In [19]:
training_DF.dropna(inplace=True)

In [23]:
sex = pd.get_dummies(training_DF['Sex'])
embark = pd.get_dummies(training_DF['Embarked'])

In [33]:
sex

,female,male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True
...,...,...
886,False,True
887,True,False
888,True,False
889,False,True


In [25]:
training_DF.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)

In [27]:
training_DF

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.000000,1,0,7.2500
1,2,1,1,38.000000,1,0,71.2833
2,3,1,3,26.000000,0,0,7.9250
3,4,1,1,35.000000,1,0,53.1000
4,5,0,3,35.000000,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,13.0000
887,888,1,1,19.000000,0,0,30.0000
888,889,0,3,29.699118,1,2,23.4500
889,890,1,1,26.000000,0,0,30.0000


In [29]:
training_DF = pd.concat([training_DF,sex,embark],axis=1)

In [31]:
training_DF

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,female,male,C,Q,S
0,1,0,3,22.000000,1,0,7.2500,False,True,False,False,True
1,2,1,1,38.000000,1,0,71.2833,True,False,True,False,False
2,3,1,3,26.000000,0,0,7.9250,True,False,False,False,True
3,4,1,1,35.000000,1,0,53.1000,True,False,False,False,True
4,5,0,3,35.000000,0,0,8.0500,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.000000,0,0,13.0000,False,True,False,False,True
887,888,1,1,19.000000,0,0,30.0000,True,False,False,False,True
888,889,0,3,29.699118,1,2,23.4500,True,False,False,False,True
889,890,1,1,26.000000,0,0,30.0000,False,True,True,False,False


In [35]:
training_DF.drop(['female','C'],axis=1,inplace=True)

In [37]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(training_DF.drop('Survived',axis=1), 
                                                    training_DF['Survived'], test_size=0.30, 
                                                    random_state=101)

In [41]:
from sklearn.linear_model import SGDClassifier

In [43]:
SGD_model = SGDClassifier(max_iter=1000, tol=1e-3, random_state=42) # tol: the stopping criterion

In [45]:
SGD_model.fit(X_train, y_train)

SGDClassifier(random_state=42)

In [47]:
SGD_predictions = SGD_model.predict(X_test)

In [49]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,SGD_predictions))

[[160   3]
 [ 98   6]]


In [75]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)   # default range is (0,1)
X_train

array([[ 1.41254721,  0.82581213, -0.92134093, ..., -1.33296126,
        -0.30519808,  0.61327497],
       [ 0.80316214, -0.36320441, -0.92134093, ..., -1.33296126,
        -0.30519808,  0.61327497],
       [-1.73203601,  0.82581213, -0.28852474, ..., -1.33296126,
        -0.30519808,  0.61327497],
       ...,
       [-0.41951432,  0.82581213,  1.2144137 , ...,  0.75020935,
        -0.30519808,  0.61327497],
       [ 0.30706019,  0.82581213,  0.00408295, ...,  0.75020935,
        -0.30519808, -1.63058993],
       [ 1.63911346, -0.36320441,  0.97710763, ..., -1.33296126,
        -0.30519808,  0.61327497]])

In [57]:
SGD_model.fit(X_train, y_train)
X_test = scaler.fit_transform(X_test)
SGD_predictions = SGD_model.predict(X_test)

In [59]:
print(confusion_matrix(y_test,SGD_predictions))

[[148  15]
 [ 51  53]]


In [69]:
from sklearn.preprocessing import MinMaxScaler   # Transforms features by scaling each feature to a given range

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)   # default range is (0,1)
X_train

array([[0.90674157, 1.        , 0.23632892, ..., 0.        , 0.        ,
        1.        ],
       [0.73146067, 0.5       , 0.23632892, ..., 0.        , 0.        ,
        1.        ],
       [0.00224719, 1.        , 0.34542479, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.37977528, 1.        , 0.60452748, ..., 1.        , 0.        ,
        1.        ],
       [0.58876404, 1.        , 0.3958696 , ..., 1.        , 0.        ,
        0.        ],
       [0.97191011, 0.5       , 0.56361653, ..., 0.        , 0.        ,
        1.        ]])

In [63]:
SGD_model.fit(X_train, y_train)
X_test = scaler.fit_transform(X_test)
SGD_predictions = SGD_model.predict(X_test)

In [65]:
print(confusion_matrix(y_test,SGD_predictions))

[[129  34]
 [ 22  82]]
